In [2]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import scipy.stats as stats 
import numpy as np 
import statsmodels.api as sm 
from sklearn.cluster import KMeans 
import scikit_posthocs as sp 
from scipy.stats import kruskal 
from sklearn.utils import resample 
from sklearn.feature_selection import RFE 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score 
 
pd.set_option('display.max_columns', None)  # Отображение всех столбцов 
# pd.set_option('display.max_rows', None)  # Ото----бражение всех строк    
regions = pd.read_csv('regions.csv') 
starts = pd.read_csv('starts.csv') 
volunteers = pd.read_csv('volunteers.csv') 
 
 
 
volunteers = volunteers[volunteers.athlete_age < 100] 
starts = starts[starts.athlete_age < 80] 
starts = starts[starts.finish_time < 5000] 
 
starts['rain.1h'] = starts['rain.1h'].fillna(0) 
starts['snow.1h'] = starts['snow.1h'].fillna(0) 
 
cnt_ludei = starts.groupby(['event_name', 'event_date', 'region']).agg( 
    cnt_runners = ('athlete_id', 'count') 
) 
 
 
 
 
ages_14 = starts[starts['athlete_age'] == 14]['athlete_id'].unique() 
ages_15 = starts[starts['athlete_age'] == 15]['athlete_id'].unique() 
common_athletes = set(ages_14) & set(ages_15) 
starts = starts[~starts['athlete_id'].isin(common_athletes)] 
starts = starts[starts['athlete_age'] > 14] 
 
 
starts['is_male'] = starts['athlete_gender'].apply(lambda x: 1 if x == 'мужской' else 0) 
 
 
volunteer_counts = volunteers.groupby(['event_date','event_name', 'volunteer_role_name']).size().unstack(fill_value=0)   
starts = starts.merge(volunteer_counts, on=['event_date','event_name'], how='left')   
 
starts = starts.dropna(subset=[col for col in starts.columns if col not in ['snow.1h', 'rain.1h']])   # удаление где 0 волонтеров в сумме было 
 
print(len(starts)) 
unique_dates = ( 
    pd.concat([ 
        starts[['athlete_id', 'event_date']], 
        volunteers[['athlete_id', 'event_date']] 
    ]) 
    .drop_duplicates() 
) 
date_counts = unique_dates.groupby('athlete_id').size().reset_index(name='unique_event_count') 
date_counts['is_repeated'] = (date_counts['unique_event_count'] > 1).astype(int) 
starts = starts.merge(date_counts[['athlete_id', 'is_repeated']], on='athlete_id', how='left') 
 
 
# starts = starts[~starts['region'].isin(['Республика Адыгея', 'Республика Карелия', 'Липецкая область', 'Сахалинская область', 'Архангельская область'])]   
starts = starts[starts.event_date <= '2023-11-15'] 
 
 
 
newbie = starts[starts['is_newbie'] == 1] 
first_runs = newbie.sort_values(by='event_date').drop_duplicates('athlete_id') 
didnt_come = first_runs[first_runs.is_repeated == 0] 
 
come = first_runs[first_runs.is_repeated == 1] 
come_with_didnt_come = pd.concat([come, didnt_come]) 
come_with_didnt_come['cnt_volunteers'] = come_with_didnt_come.iloc[:, 25:52].sum(axis=1) 
come_with_didnt_come = come_with_didnt_come.merge(cnt_ludei,on=['event_name', 'event_date', 'region'], how="left") 
print(len(come_with_didnt_come))


166516
18114


In [5]:
come_with_didnt_come

,record_id,athlete_id,event_name,event_date,city,region,finish_time,athlete_gender,athlete_age,is_newbie,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,weather.main,weather.description,rain.1h,snow.1h,track_description_surface,is_male,Ведущий велосипед,Завершение мероприятия,Замыкающий,Инструктаж новых участников,Координатор парковки,Координация волонтёров,Лидер для слабовидящих,Маршал,Обработка результатов,Организатор,Организация финиша,Пейсер,Подготовка мероприятия,Помощь в раздаче карточек позиций,Проведение разминки,Проверка карточек позиций,Проверка трассы,Раздача карточек позиций,Разное,Связи с общественностью,Секундомер,Сканирование штрих-кодов,Сортировка карточек,Составление отчёта,Сурдопереводчик,Фотограф,Хранение и доставка оборудования,is_repeated,cnt_volunteers,cnt_runners
0,4790,790069326,Елагин остров,2023-01-01,Санкт-Петербург,Санкт-Петербург,1837,женский,38,1,3.94,-1.20,996,88,2.14,100,8.00,260,Clouds,overcast clouds,0.0,0.00,смешанное,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,1,15.0,41
1,4816,790126902,Петергоф Александрийский,2023-01-01,Санкт-Петербург,Санкт-Петербург,3371,женский,39,1,2.71,-1.99,997,95,1.99,100,5.95,263,Clouds,overcast clouds,0.0,0.00,грунт,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,4.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,3.0,0.0,1,20.0,55
2,4782,790126900,Волгоград панорама,2023-01-01,Волгоград,Волгоградская область,1723,женский,37,1,1.74,-3.68,1025,87,-0.16,75,7.00,260,Clouds,broken clouds,0.0,0.00,твердое,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1,13.0,30
3,4816,790126898,Петергоф Александрийский,2023-01-01,Санкт-Петербург,Санкт-Петербург,1861,женский,17,1,2.71,-1.99,997,95,1.99,100,5.95,263,Clouds,overcast clouds,0.0,0.00,грунт,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,4.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,3.0,0.0,1,20.0,55
4,4822,790126893,Раменское Городской парк,2023-01-01,Раменское,Московская область,2380,мужской,52,1,1.29,-2.06,1008,100,1.29,100,3.13,180,Drizzle,light intensity drizzle,0.0,0.22,твердое,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,1,11.0,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18109,12484,790146760,Сосновка,2023-11-11,Санкт-Петербург,Санкт-Петербург,2179,женский,33,1,5.73,3.50,1007,92,4.53,100,2.84,186,Clouds,overcast clouds,0.0,0.00,грунт,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,1.0,0.0,0.0,1.0,1.0,0,16.0,80
18110,12576,790148734,Сириус,2023-11-11,Сочи,Краснодарский край,1285,мужской,32,1,16.41,15.23,1017,43,3.81,75,7.00,40,Clouds,broken clouds,0.0,0.00,беговая дорожка,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0,10.0,35
18111,12427,790148849,Мещерское озеро,2023-11-11,Нижний Новгород,Нижегородская область,1558,мужской,46,1,1.67,-2.23,1019,100,1.67,100,4.00,190,Fog,fog,0.0,0.00,твердое,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,2.0,2.0,1.0,0.0,0.0,0.0,1.0,2.0,0,18.0,30
18112,12557,790147042,Интернационалистов,2023-11-11,Санкт-Петербург,Санкт-Петербург,1385,мужской,40,1,6.13,6.13,1004,92,4.93,100,0.00,0,Clouds,overcast clouds,0.0,0.00,твердое,1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,2.0,1.0,2.0,4.0,2.0,1.0,1.0,0.0,0.0,2.0,3.0,0,27.0,47


Q критерий какого хрена

In [14]:
import numpy as np


def cochran_q_test(data, alpha=0.05):
    variances = [np.var(group, ddof=1) for group in data]
    

    max_variance = max(variances)
    sum_variances = sum(variances)
    Q = max_variance / sum_variances
    

    k = len(data)
    
    from scipy.stats import f
    Q_crit = f.ppf(1 - alpha, dfn=1, dfd=k - 1) / (f.ppf(1 - alpha, dfn=1, dfd=k - 1) + k - 1)
    
    result = "Однородны" if Q <= Q_crit else "Неоднородны"
    
    return Q, Q_crit, result



Q, Q_crit, result = cochran_q_test(data)
print(f"Q: {Q:.4f}, Критическое значение: {Q_crit:.4f}, Результат: {result}")


Q: 0.3333, Критическое значение: 0.9025, Результат: Однородны


In [ ]:
import pandas as pd
import numpy as np

for col in come_with_didnt_come.columns[25:51]:
    come_with_didnt_come[f'has_{col}'] = (come_with_didnt_come[col] > 0).astype(int)

def cochran_q_for_proportions(data, variable, success_col, alpha=0.05):

    # Разделяем выборки
    group1 = data[data[variable] == 1][success_col]
    group2 = data[data[variable] == 0][success_col]
    
    # Вычисляем доли успехов
    p1 = group1.mean()
    p2 = group2.mean()
    
    # Вычисляем дисперсии долей
    var1 = p1 * (1 - p1) / len(group1)
    var2 = p2 * (1 - p2) / len(group2)
    
    # Вычисляем Q
    variances = [var1, var2]
    Q = max(variances) / sum(variances)
    
    # Количество групп
    k = 2
    
    # Критическое значение Q
    from scipy.stats import f
    Q_crit = f.ppf(1 - alpha, dfn=1, dfd=k - 1) / (f.ppf(1 - alpha, dfn=1, dfd=k - 1) + k - 1)
    
    # Проверка гипотезы
    result = "Однородны" if Q <= Q_crit else "Неоднородны"
    
    return Q, Q_crit, result

# 3. Применение к каждому волонтеру
results = []
for col in come_with_didnt_come.columns[25:51]:
    variable = f'has_{col}'
    Q, Q_crit, result = cochran_q_for_proportions(come_with_didnt_come, variable, success_col='is_repeated')  # Замените success на нужный столбец
    results.append({'volunteer': col, 'Q': Q, 'Q_crit': Q_crit, 'result': result})

# Преобразуем результаты в DataFrame
results_df = pd.DataFrame(results)

results_df.to_csv("cochran_q_test_results.csv", index=False)

# Выводим первые строки для проверки
print(results_df)

                            volunteer         Q    Q_crit       result
0              Завершение мероприятия  0.876678  0.993844    Однородны
1                          Замыкающий  0.987676  0.993844    Однородны
2         Инструктаж новых участников  0.615188  0.993844    Однородны
3                Координатор парковки  0.992963  0.993844    Однородны
4              Координация волонтёров  0.800095  0.993844    Однородны
5              Лидер для слабовидящих  0.981342  0.993844    Однородны
6                              Маршал  0.679713  0.993844    Однородны
7               Обработка результатов  0.767294  0.993844    Однородны
8                         Организатор  0.996197  0.993844  Неоднородны
9                  Организация финиша  0.832663  0.993844    Однородны
10                             Пейсер  0.896849  0.993844    Однородны
11             Подготовка мероприятия  0.708517  0.993844    Однородны
12  Помощь в раздаче карточек позиций  0.835597  0.993844    Однородны
13    